# Наш любимый парсер

Тут я накидал примерный каркас нашего будущего скачивателя ссылок. 

__Задание:__ Написать парсер для скачки ссылок и собрать все ссылки на недвижимость с ЦИАН.

* Пытайтесь писать код в виде маленьких функций. Каждая функция делает что-то полезное. Из них собирайте основной цикл с условиями.
* Не бойтесь экспериментировать, рассказывайте про свои эксперименты другим ребятам в чате. Пульте свой код в свою ветку. Делитесь своими лучшими функциями с другими. В мастер нельзя заливать свои функции без ревью. Делать ревью научимся, когда появится первый желающий залить свой код в мастер. Будем заниматься этим через пул-реквесты. 

* Если удалось скачать все ссылки, начинайте писать код для скачки их внутренностей. Снова много мелких функций для вытаскивания каждого элемента. Мой код с подготовки пары может вам помочь. Там много чего написано. Только там одна большая уродливая функция. Это не очень хорошо. 


Если у вас нет каких-то пакетов, ставьте их через `pip` либо `conda`: 

Если хотите ставить их прямо через юпитер, дописывайте перед командой `!`. Это сигнал, что надо обратиться к консоли. 

In [3]:
import requests   # пакет для парсинга
import time       # пакет для работы с временем
import pickle     # поможет сохранять данные

from bs4 import BeautifulSoup   # пакет для работы с html-деревом
from tqdm import tqdm_notebook  # пакет для прогресса в циклах 

from collections import defaultdict  # удобные словари смотри: https://python-scripts.com/import-collections

Этот блок кода сформирует для нас ссылки, по которым мы в дальнейшем будем качать квартиры. 

In [2]:
districts = [1, 325, 326, 4, 5, 6, 7, 8, 9, 10, 11, 151]

rooms = ['room1=1', 'room2=1', 'room3=1', 'room4=1', 
         'room5=1', 'room6=1', 'room7=1', 'room9=1']

objt = [1,2]

def create_url(params):
    main_page = 'https://www.cian.ru/cat.php?'
    params_1 = 'engine_version=2&deal_type=sale&offer_type=flat'
    params_2 = '&district={district}&{room}&region={region}&object_type={objt}'.format(**params)
    return main_page + params_1 + params_2

main_urls = [ ]

for dis in districts:
    for rm in rooms:
        for ob in objt:
            params = {
                'region': 1,
                'district': dis,
                'room': rm,
                'objt': ob}
            
            main_urls.append(create_url(params))
            
len(main_urls) # Ссылки, по которым надо пройтись

192

Небольшие вспомогательные функции. Перед тем, как использовать их, потестите их! 

In [4]:
def is_capcha(soup):
    """
        Проверяет вылезла ли капча

        soup: bs4
            html-страничка, прогнанная через Beautifulsoap
    """
    
    if soup.title.text == 'Captcha - база объявлений ЦИАН':
        return True
    else:
        return False
    
    
def is_empty(soup):
    """
        Проверяет есть ли на странице что скачивать или она пустая с p=1

        soup: bs4
            html-страничка, прогнанная через Beautifulsoap
    """
    
    # нужно бы её написать :) 
    
    ...

# Любые другие полезные мелкие функции

Разные функции для скачки данных (работают в разных режимах, например) 

In [6]:
def get_soup(url, p):
    """
        Качает по ссылке url и номеру страницы p её содержимое, отдаёт в виде bs4
    """    
    
    url = url + '&p={}'.format(p)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content)
    return soup 


def get_soup_retry(url, p, MAX_RETRIES=10):
    """
        Качает по ссылке urlи номеру страницы p её содержимое, отдаёт в виде bs4
        Пытается избежать Disconection и делает по MAX_RETRIES=10 попыток при проблемах
    """
    
    url = url + '&p={}'.format(p)
    session = requests.Session()
    
    adapter = requests.adapters.HTTPAdapter(max_retries=MAX_RETRIES)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    
    resp = session.get(url)
    soup = BeautifulSoup(resp.content)
    return soup

Функции, которые расфасовывают ссылки.

In [8]:
def get_hrefs(soup):
    """
        Находит в soup все ссылки на дома (осторожно с классами)
    """
    s = soup.find_all('div', {'class': '_93444fe79c--card--_yguQ'})
    
    hrefs = [item.find('a', {'class': 'c6e8ba5398--header--1fV2A'}).get("href") 
                         for item in s]
    return hrefs    

Протестируем:

In [10]:
url = main_urls[0]

soup1 = get_soup(url,1)
soup2 = get_soup_retry(url,1)

print(is_capcha(soup1))
print(is_capcha(soup2))

hrefs = get_hrefs(soup1)
hrefs[:5]

False
False


['https://www.cian.ru/sale/flat/226177506/',
 'https://www.cian.ru/sale/flat/225538051/',
 'https://www.cian.ru/sale/flat/225446506/',
 'https://www.cian.ru/sale/flat/225845779/',
 'https://www.cian.ru/sale/flat/226150729/']

Основной контур для скачки ссылок. Будем класть их в словарик по каждому району. 

```
{ 
    ссылка на район 1:   [лист из ссылок на дома], 
    ссылка на район 2:   [лист из ссылок на дома],
  ....
 
}

```

Такая структура хранения данных поможет нам потом легко понять по каким районам ссылки плохо скачались. 

In [12]:
hrefs_dict = defaultdict(list)  # словарик под ссылки 

In [ ]:
# основной цикл идёт по ссылкам на районы
for url in tqdm_notebook(main_urls):
    
    hrefs = []  # создали для текущего района пустой массив под ссылки на дома
    
    for p in range(1, 55):
        
        time.sleep(5)           # между запросами спим по 5 секунд для приличия (много!)
        soup = get_soup(url, p)  # скачали данные 
        
        # пока вылезает капча, отдыхаем по 5 минут 
        while is_capcha(soup):
            print('Капча вылезла :(')
            time.sleep(60*5)
            soup = get_soup(url, p)  # внеочередная попытка пробится сквозь капчу 
            
        
        cur_hrefs = get_hrefs(soup)  # вытащили ссылки 
        
        # проверяем всё ли скачали и пошли ли дубли
        if p > 1:
            if cur_hrefs[-1] == hrefs[-1]:
                break
                
        hrefs.extend(cur_hrefs) # если нет дублей, обновили вектор с ссылками 
        
    # внутренний цикл кончился, кладём для текущего района, url, кваритиры в словарик
    hrefs_dict[url] = hrefs
    

Если мерзкий код падает и обижает нас, сохраняем промежуточный результат и качаем дальше. 

In [ ]:
# осторожнее с именами файлов! случайно можно перезаписать готовый :( 

with open('data/data.pickle', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
# можно проверить всё ли ок сохранилось
with open('data/data.pickle', 'rb') as f:
    data_new = pickle.load(f)